In [18]:
# Environment setup and module import
from __future__ import print_function
import torch
import torch.optim as optim
import torch.nn as nn
import pandas as pd

device = 'cuda' if torch.cuda.is_available() else 'cpu'
dtype = torch.float

False


In [12]:
# Import DF from CSV

train_file = 'data/mini_cleaned.csv'
train_df = pd.read_csv(train_file, low_memory=False)



In [ ]:
# Perform whatever conversions are necessary to get everything into float format.

# Perform standardization for input parameters



In [19]:
# Pytorch setup

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 81, 1000, 100, 2

# Create random input and output data
# To do: Create input data, once I have the paramters mapped
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

# To do: figure out appropriate learning rate
learning_rate = 1e-5

#x_train_tensor = torch.from_numpy(x_train).float().to(device)
#y_train_tensor = torch.from_numpy(y_train).float().to(device)